In [1]:
import dash
from dash import dcc, html
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

# Load the dataset from the 'data' sub-folder
df = pd.read_csv('data/CFB_TransferPortal23.csv', encoding='ISO-8859-1')
df

,Data,Position,Conference,Conference Type,Receiving Team,Player,From School
0,QB Thomas Castellanos (from UCF),QB,ACC,Power 5,Boston College,Thomas Castellanos,UCF
1,RB Kye Robichaux (from Western Kentucky),RB,ACC,Power 5,Boston College,Kye Robichaux,Western Kentucky
2,WR Ryan O'Keefe (from UCF),WR,ACC,Power 5,Boston College,Ryan O'Keefe,UCF
3,OL Kyle Hergel (from Texas State),OL,ACC,Power 5,Boston College,Kyle Hergel,Texas State
4,OL Logan Taylor (from Virginia),OL,ACC,Power 5,Boston College,Logan Taylor,Virginia
...,...,...,...,...,...,...,...
1831,DB Isaiah Rutherford (from Arizona),DB,Independents,Non Power 5,UMass,Isaiah Rutherford,Arizona
1832,DB Dashaun Jerkins (from Ole Miss),DB,Independents,Non Power 5,UMass,Dashaun Jerkins,Ole Miss
1833,DB Jerrod Cameron (from Coastal Carolina),DB,Independents,Non Power 5,UMass,Jerrod Cameron,Coastal Carolina
1834,DB Steven Ortiz (from Minnesota),DB,Independents,Non Power 5,UMass,Steven Ortiz,Minnesota


In [2]:
df_count = df.groupby(['From School', 'Receiving Team']).size().reset_index(name='counts')


In [3]:
import holoviews as hv
from holoviews import opts, dim
from holoviews.plotting.links import Chord

hv.extension('bokeh')

# Define the Chord data
chords = Chord(df_count).select(value=(1, None))

# Plot
chords.opts(
    opts.Chord(cmap='Category10', edge_cmap='Category10', edge_color=dim('From School').str(), 
               labels='To School', node_color=dim('index').str())
)


ModuleNotFoundError: No module named 'holoviews'

In [ ]:
#power5_df = df[df['Conference'] == 'Big 12']

In [6]:
# Group by 'From School' and 'Receiving Team' to get the transfer counts
From_grouped = df.groupby(['From School', 'Receiving Team']).size().reset_index(name='count')
From_grouped = grouped.sort_values(by='count', ascending=False)

From_grouped

,From School,Receiving Team,count
551,Jackson State,Colorado,10
517,Incarnate Word,Texas State,9
694,Maryland,Charlotte,7
54,Arizona,UMass,7
733,Miami,SMU,6
...,...,...,...
1530,Washington State,Sam Houston,1
1529,Washington State,Oklahoma State,1
1509,Wake Forest,Marshall,1
1510,Wake Forest,Notre Dame,1


In [7]:
# Group by 'From School' and 'Receiving Team' to get the transfer counts
To_grouped = df.groupby(['Receiving Team','From School']).size().reset_index(name='count')
To_grouped = grouped.sort_values(by='count', ascending=False)

To_grouped

,From School,Receiving Team,count
551,Jackson State,Colorado,10
517,Incarnate Word,Texas State,9
694,Maryland,Charlotte,7
54,Arizona,UMass,7
733,Miami,SMU,6
...,...,...,...
1530,Washington State,Sam Houston,1
1529,Washington State,Oklahoma State,1
1509,Wake Forest,Marshall,1
1510,Wake Forest,Notre Dame,1


In [8]:
# Create a list of all unique schools and teams
labels = list(set(pd.concat([grouped['From School'], grouped['Receiving Team']])))

# Convert 'From School' and 'Receiving Team' to indices based on 'labels'
source = [labels.index(school) for school in grouped['From School']]
target = [labels.index(team) for team in grouped['Receiving Team']]

# Create the Sankey diagram
fig = go.Figure(grouped=[go.Sankey(
    node=dict(
        pad=15,
        thickness=20,
        line=dict(color="black", width=0.5),
        label=labels
    ),
    link=dict(
        source=source,
        target=target,
        value=grouped['count']
    )
)])

fig.update_layout(title_text="Sankey Diagram of Transfers", font_size=10)
fig.show()


TypeError: invalid Figure property: grouped
grouped
Bad property path:
grouped
^^^^^^^

In [54]:

# Extract unique schools and teams for nodes
all_nodes = list(set(grouped['From School']).union(set(grouped['Receiving Team'])))
grouped.head(10)

,From School,Receiving Team,count
551,Jackson State,Colorado,10
517,Incarnate Word,Texas State,9
694,Maryland,Charlotte,7
54,Arizona,UMass,7
733,Miami,SMU,6
391,Florida State,Colorado,5
192,California,Nevada,5
1535,Weber State,BYU,4
745,Michigan,Charlotte,4
1049,Oregon,Nevada,4


In [55]:

# Map 'From School' and 'Receiving Team' to their respective indices in the node list
grouped['source'] = grouped['From School'].apply(lambda x: all_nodes.index(x))
grouped['target'] = grouped['Receiving Team'].apply(lambda x: all_nodes.index(x))

# Generate the Sankey diagram
sankey_fig = go.Figure(data=[go.Sankey(
    node=dict(
        pad=15,
        thickness=20,
        line=dict(color="black", width=0.5),
        label=all_nodes
    ),
    link=dict(
        source=grouped['source'],
        target=grouped['target'],
        value=grouped['count']
    ))])

sankey_fig.update_layout(title_text="Transfers from 'From School' to 'Receiving Team' (Power 5)", font_size=10)
sankey_fig.show()


In [56]:
# Build a list of unique schools to act as the nodes in the Sankey diagram
unique_schools = pd.concat([df['From School'], df['Receiving Team']]).unique()
school_dict = {school: i for i, school in enumerate(unique_schools)}

# Populate the source, target and value lists with data from the DataFrame
for index, row in df.iterrows():
    sources.append(school_dict[row['From School']])
    targets.append(school_dict[row['Receiving Team']])
    values.append(row['count'])

# Create the Sankey diagram
fig = go.Figure(data=[go.Sankey(
    node=dict(
        pad=15,
        thickness=20,
        line=dict(color='black', width=0.5),
        label=list(unique_schools)
    ),
    link=dict(
        source=grouped['source'],
        target=grouped['target'],
        value=grouped['count']
    )
)])

fig.update_layout(title_text='Player Transfers Between Schools', font_size=10)
fig.show()


NameError: name 'sources' is not defined

In [12]:
# Create a count of the relationships
df_count = df.groupby(['Recieving Team', 'From School']).size().reset_index(name='counts')

KeyError: 'Recieving Team'